# Dynamic Growth Model

## Imports

In [ ]:
import os
import random
import sys
import pickle
from functools import partial


import matplotlib.pyplot as plt
import numpy as np

from core.generator import gen_u_daily_sine
from core.lettuce_model import lettuce_growth_model, get_default_constants
from core.plot import plot_response
from core.simulate import simulate

# For reproducibility
random.seed(40)

# Enable automatic module reload
%load_ext autoreload
%autoreload 2

## Setup Environment in Google Colab

In [ ]:
# Setup Google Colab Environment
if "google.colab" in sys.modules:
    from google.colab import drive, userdata

    drive.mount("/content/drive")
    os.chdir(userdata.get("project_path"))
    !git pull
    !git checkout dev
    !git pull
    %pip install -r requirements-dev.txt
    !pre-commit install --hook-type pre-push

In [ ]:
# Commit changes
if "google.colab" in sys.modules:
    !git add -A
    name = userdata.get("github_name")
    email = userdata.get("github_email")
    !git -c user.name="$name" user.email="$email" commit -m "FIX: Githu"

In [ ]:
# Push changes to GitHub
if "google.colab" in sys.modules:
    token = userdata.get("github_token")
    !git push https://$token@github.com/MarekWadinger/dynamic_opt_growth_model.git

## Case Study: reuse of model with selected constants

In [ ]:
x0 = [0.72, 2.7]
u_min = [10, 0, 400]
u_max = [25, 100, 800]
sim_time = 1200

In [ ]:
u = partial(gen_u_daily_sine, u_min=u_min, u_max=u_max)

t_out, y_out, u_out = simulate(
    lettuce_growth_model, u=u, sim_time=sim_time, t_s=60 * 60, x0=x0
)

In [ ]:
plot_response(t_out, y_out, u_out, u_min=u_min, u_max=u_max)
plt.show()

# How to Change Constants?

In [ ]:
from core import lettuce_model

constants = {
    key: value for key, value in vars(lettuce_model).items() if key.isupper()
}
constants_new = {
    "C_YF": 3,
    "C_GR_MAX": 5e-6,
    "C_K": 1.2,
    "C_LAR": 75e-3,
    "C_EPSILON": 17e-6,
    "UF": 0.5,
}
constants.update(constants_new)

In [ ]:
u = partial(gen_u_daily_sine, u_min=u_min, u_max=u_max)

t_out, y_out2, u_out = simulate(
    lettuce_growth_model,
    u=u,
    sim_time=sim_time,
    t_s=60 * 60,
    x0=x0,
    **constants,
)
plot_response(t_out, y_out2, u_out, u_min=u_min, u_max=u_max)
plt.show()

## One parameter random generating data

In [ ]:
HOW_MANY = 100  # how many values to generate


def randomize_const(
    consts: dict[str, float],
    which: list | str | None = "all",
    min_change: float = 0.9,
    max_change: float = 0.1,
    seed: int | None = None,
):
    """Multiply value in the dictionary by random number.

    Args:
        consts (dict[str, float], optional): Dictionary with constants. Defaults to None.
        which (str, optional): Which constant to change. Defaults to None.
        min_change (float, optional): Minimum change in percentage. Defaults to 0.9.
        max_change (float, optional): Maximum change in percentage. Defaults to 0.1.

    Returns:
        dict[str, float]: Dictionary with changed constant or new dictionary .
    """
    np.random.seed(seed)

    if which is None:
        # This will give us the only key if the dictionary has only one
        keys = [random.choice(list(consts.keys()))]
    elif which == "all":
        keys = list(consts.keys())
    elif isinstance(which, str):
        keys = [which]

    for key in keys:
        consts[key] = random.uniform(min_change, max_change) * consts[key]

    return consts

In [ ]:
def simulate_randomized(
    which: str, how_many: int = HOW_MANY
) -> dict[str, np.ndarray]:
    c_list: list[dict] = [
        randomize_const(get_default_constants(), which)
        for _ in range(how_many)
    ]

    results = {}

    for c in c_list:
        _, y_out, _ = simulate(
            lettuce_growth_model,
            u=u,
            sim_time=sim_time,
            t_s=60 * 60,
            x0=x0,
            **c,
        )
        results[f"{c}"] = y_out

    return results


def save_results(
    results: dict[str, np.ndarray], which: str, how_many: int = HOW_MANY
):
    with open(
        f"results/constant_estimation/{which.lower()}_{how_many}.pkl",
        "wb",
    ) as f:
        pickle.dump(results, f)

In [ ]:
for const in ["C_YF", "C_GR_MAX", "C_K", "C_LAR", "C_EPSILON"]:
    print(f"Simulating {HOW_MANY} with random {const:.<12}", end="\r")
    results = simulate_randomized(const)

    save_results(results, const)

## Every parameter random generating data

In [ ]:
which = "all"

results = simulate_randomized(which)

save_results(results, which)